<img src="unicamp.png" width="150" height="150">

# Question and Answering with SimpleTransformers

## Read Data

In [ ]:
import json

FILE_INPUT = 'results/dataset.json'

with open(FILE_INPUT, 'r') as f:
    train_data = json.load(f)

train_data = [item for topic in train_data['data'] for item in topic['paragraphs']]

## Get Model

In [ ]:
import torch
from simpletransformers.question_answering import QuestionAnsweringModel

In [ ]:

# train_args = {
#     'learning_rate': 3e-5,
#     'num_train_epochs': 2,
#     'max_seq_length': 384,
#     'doc_stride': 128,
#     'overwrite_output_dir': True,
#     'reprocess_input_data': False,
#     'train_batch_size': 2,
#     'gradient_accumulation_steps': 8,
# }

# model = QuestionAnsweringModel('bert', 'bert-base-cased', args=train_args)
model = QuestionAnsweringModel('bert', 'bert-base-cased')

## Train Model

In [ ]:
model.train_model(train_data)

In [ ]:
torch.save(model, 'bert_qea')

## Experiment